# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [44]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [45]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [46]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [47]:
a

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

In [48]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [49]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [50]:
np.linalg.matrix_rank(a1)

6

In [51]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [52]:
def non_zero_minor(a):
    N = a.shape[0]
    c = 0
    for k in range(N+1):
        b = np.zeros((k, k), dtype=float) #матрица первых k строк и k столбцов
        for i in range(k):
            for j in range(k):
                b[i,j] = a[i,j]
        if np.linalg.det(b) == 0: #проверяем равенство нулю
            с = c + 1
            return 'There is zero minor'
    if c == 0:
        return 'All leading minors are non-zero'

Проверка для матриц `a` и `a1`:

In [53]:
non_zero_minor(a)

'All leading minors are non-zero'

In [54]:
non_zero_minor(a1)

'There is zero minor'

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [55]:
def diy_lu_pivot(a):
    
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    
    for j in range(N-1):
        
        #Поиск максимального элемента столбца (partial pivoting)
        c = np.zeros((1, N-j))
        for i in range(N-j):
            c[0,i] = u[i+j,j] #Берем столбец с номером j
        i_max = np.argmax(abs(c)) + j #Номер строки с максимальным элементом
        
        #Составляем матрицу перестановки для шага j
        p = np.eye(N)
        p[j], p[i_max] = p[i_max], p[j].copy() #строка с максимальным элементом меняется на верхнюю рассматриваемую
        
        u = p@u

        P = p@P #Собираем матрицу перестановки

        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam 
        
    return P, L, u

In [56]:
# Разложение для матрицы а
P, L, u = diy_lu_pivot(a)

print(P, "\n")
print(L, "\n")
print(u, "\n")

print(L@u - P@a)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.727 1.    0.    0.    0.   ]
 [1.    0.857 0.41  1.    0.    0.   ]
 [1.    0.941 0.178 0.426 1.    0.   ]
 [1.    0.5   0.706 0.835 0.789 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  1.110e-16 -2.317e-17 -1.999e-19 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.431e-16  6.463e-18  1.577e-19  0.000e+00  6.730e-06]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]
 [ 0.000e+00 -5.114e-01 -5.844e-01 -6.136e-01 -6.294e-01 -6.392e-01]
 [ 0.000e+

In [57]:
# Разложение для матрицы а1
P, L, u = diy_lu_pivot(a1)

print(P, "\n")
print(L, "\n")
print(u, "\n")

print(L@u - P@a1)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.727 1.    0.    0.    0.   ]
 [1.    0.857 0.088 1.    0.    0.   ]
 [1.    0.941 0.038 0.208 1.    0.   ]
 [1.    0.    0.151 0.514 0.641 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.776e-17 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  1.110e-16 -1.427e-17 -4.160e-18  1.841e-03  3.821e-03]
 [ 0.000e+00 -1.267e-16  3.391e-18  2.669e-18  0.000e+00 -1.233e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]
 [ 0.000e+00 -1.636e+00 -1.870e+00 -1.964e+00 -2.014e+00 -2.045e+00]
 [ 0.000e+

In [58]:
def a_from_LUP(P,L,u):
    return np.linalg.inv(P)@L@u

In [59]:
P, L, u = diy_lu_pivot(a)
b = a_from_LUP(P,L,u)
print(b)
print(b-a)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.364 0.779 0.458 0.253 0.111]
 [3.    1.071 0.652 0.473 0.375 0.313]
 [3.    0.882 0.517 0.366 0.283 0.23 ]
 [3.    1.875 1.467 1.262 1.138 1.055]
 [3.    0.75  0.429 0.3   0.231 0.188]]
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -5.114e-01 -5.844e-01 -6.136e-01 -6.294e-01 -6.392e-01]
 [ 0.000e+00 -2.922e-01 -2.302e-01 -1.794e-01 -1.427e-01 -1.156e-01]
 [ 0.000e+00 -1.891e-01 -1.349e-01 -1.029e-01 -8.297e-02 -6.956e-02]
 [ 0.000e+00  9.926e-01  9.495e-01  8.960e-01  8.550e-01  8.243e-01]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]


In [60]:
P1, L1, u1 = diy_lu_pivot(a1)
b1 = a_from_LUP(P1,L1,u1)
print(b1)
print(b1-a1)

[[ 3.     3.     3.     3.     3.     3.   ]
 [ 3.     1.364 -0.506 -0.892 -1.132 -1.295]
 [ 3.     1.071  0.652  0.424  0.292  0.206]
 [ 3.     0.882  0.517  0.366  0.284  0.232]
 [ 3.     3.     2.752  2.661  2.605  2.567]
 [ 3.     0.75   0.429  0.3    0.231  0.188]]
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -1.636e+00 -1.870e+00 -1.964e+00 -2.014e+00 -2.045e+00]
 [ 0.000e+00 -2.922e-01 -2.302e-01 -2.283e-01 -2.253e-01 -2.226e-01]
 [ 0.000e+00 -1.891e-01 -1.349e-01 -1.029e-01 -8.217e-02 -6.785e-02]
 [ 0.000e+00  2.118e+00  2.235e+00  2.295e+00  2.322e+00  2.336e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]
